In [ ]:
import spacy
import pandas as pd
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

In [ ]:
import pandas as pd

def read_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return pd.DataFrame({'text': f.read().splitlines()})

test_data = read_lines('/content/wiki.test.tokens')
train_data = read_lines('/content/wiki.train.tokens')
valid_data = read_lines('/content/wiki.valid.tokens')

print(test_data.head())
print(train_data.head())
print(valid_data.head())


                                                text
0                                                   
1                                  = Robert <unk> = 
2                                                   
3   Robert <unk> is an English film , television ...
4   In 2006 , <unk> starred alongside <unk> in th...
                                                text
0                                                   
1                       = Valkyria Chronicles III = 
2                                                   
3   Senjō no Valkyria 3 : <unk> Chronicles ( Japa...
4   The game began development in 2010 , carrying...
                                                text
0                                                   
1                              = Homarus gammarus = 
2                                                   
3   Homarus gammarus , known as the European lobs...
4                                                   


In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
from string import punctuation
def clean_text(text):
 analyzed_text=nlp(text.lower())
 filtered_text=[
     tokens.text for tokens in analyzed_text if tokens.text not in punctuation
     ]
 return ' '.join(filtered_text)

In [ ]:
train_data["cleaned"]=train_data['text'].apply(clean_text)
test_data["cleaned"]=test_data['text'].apply(clean_text)
valid_data["cleaned"]=valid_data['text'].apply(clean_text)

In [ ]:
# 1. Prepare corpus (list of cleaned sentences)
corpus = train_data['cleaned'].tolist()

# 2. Tokenizer: fit on corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index) + 1

# 3. Create sequences
sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(2, len(token_list)+1):
        seq = token_list[:i]
        sequences.append(seq)

# 4. Pad sequences
max_len = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

# 5. Split sequences into inputs and labels
X = sequences[:, :-1]
y = sequences[:, -1]

# 6. One-hot encode labels
y = np.eye(vocab_size)[y]

# 7. Build LSTM model
model = Sequential([
    Embedding(vocab_size, 10, input_length=max_len-1),
    LSTM(50),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 8. Train model
model.fit(X, y, epochs=10, batch_size=64)

# Print summary
model.summary()


In [ ]:
import spacy
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
from string import punctuation

# Load data
def read_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return pd.DataFrame({'text': f.read().splitlines()})

test_data = read_lines('/content/wiki.test.tokens')
train_data = read_lines('/content/wiki.train.tokens')
valid_data = read_lines('/content/wiki.valid.tokens')

# Clean text
nlp = spacy.load('en_core_web_sm')
def clean_text(text):
    analyzed_text = nlp(text.lower())
    return ' '.join([token.text for token in analyzed_text if token.text not in punctuation])

train_data["cleaned"] = train_data['text'].apply(clean_text)
test_data["cleaned"] = test_data['text'].apply(clean_text)
valid_data["cleaned"] = valid_data['text'].apply(clean_text)

# Prepare corpus
corpus = train_data['cleaned'].tolist()

# Tokenizer with limited vocab size
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
vocab_size = min(max_words, len(word_index)) + 1

# Create sequences
sequences = []
for line in corpus[:10000]:  # Limit lines to avoid memory issues
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(2, len(token_list)+1):
        seq = token_list[:i]
        sequences.append(seq)

# Pad sequences
max_len = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

# Split into X and y
X = sequences[:, :-1]
y = sequences[:, -1]

# No need to one-hot encode y
# Use sparse_categorical_crossentropy instead
model = Sequential([
    Embedding(vocab_size, 50, input_length=max_len-1),
    LSTM(64),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X, y, epochs=5, batch_size=64)

# Summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1697/7290 ━━━━━━━━━━━━━━━━━━━━ 28:13 303ms/step - accuracy: 0.0743 - loss: 7.0610

KeyboardInterrupt: 